In [ ]:
from pathlib import Path
from typing import Optional

import plotly.express as px
import plotly.graph_objects as go
import polars as pl

from plotly.subplots import make_subplots

from afft.utils.log import logger

In [ ]:
INPUT_DIR: Path = Path("/media/martin/barracuda/acfr_revisits_cameras")
OUTPUT_DIR: Path = Path("/media/martin/barracuda/acfr_revisits_cameras")

# Visits that need updated latitude / longitude
# r29mrd5h_20090612_225306_cameras
# r29mrd5h_20090613_100254_cameras
# r29mrd12_20090613_010853_cameras
# r29mrd12_20090613_104954_cameras

FILENAME: str = "r29mrd12_20090613_104954_cameras.csv"

camera_paths: dict = dict()
camera_paths["renav"]: Path = INPUT_DIR / Path(
    f"acfr_cameras_renav_filtered/{FILENAME}"
)
camera_paths["squidle"]: Path = INPUT_DIR / Path(f"acfr_cameras_squidle/{FILENAME}")
output_path: Path = OUTPUT_DIR / Path(f"acfr_cameras_squidle_filtered/{FILENAME}")

for key in ["renav", "squidle"]:
    if not camera_paths[key].exists():
        logger.error(f"file does not exist: {camera_paths[key]}")

In [ ]:
cameras: dict = dict()
cameras["renav"]: pl.DataFrame = pl.read_csv(camera_paths["renav"])
cameras["squidle"]: pl.DataFrame = pl.read_csv(camera_paths["squidle"])

figs: dict = dict()

logger.info(cameras["renav"].head())
logger.info(cameras["squidle"].head())

In [ ]:
# TODO: Overwrite latitude / longitude values in renav where renav.label == squidle.key
cam_r: pl.DataFrame = cameras["renav"]

# Filter squidle cameras that are not among the renav cameras
cameras["filtered"]: pl.DataFrame = cameras["squidle"].filter(
    pl.col("key").is_in(cameras["renav"]["stereo_left_label"])
)

if len(cameras["filtered"]) != len(cameras["renav"]):
    logger.error(
        "number of cameras filtered from Squidle does not match the number of cameras from Renav"
    )
else:
    logger.info(f"Camera count: {cameras['filtered']}")

# Sort cameras by label
cameras["filtered"] = cameras["filtered"].sort("key")
cameras["renav"] = cameras["renav"].sort("stereo_left_label")


# Replace lat/lon in Renav cameras with Squidle cameras
cameras["updated"]: pl.DataFrame = cameras["renav"].with_columns(
    [
        cameras["filtered"]["pose.lat"].alias("latitude"),
        cameras["filtered"]["pose.lon"].alias("longitude"),
    ]
)

### Define plot functions

In [ ]:
def plot_attitudes(cameras: pl.DataFrame, show: bool = True) -> None:
    """TODO"""
    fig = make_subplots(rows=3, cols=1, subplot_titles=("Roll", "Pitch", "Heading"))

    fig.append_trace(
        go.Scatter(
            x=cameras["timestamp"],
            y=cameras["pitch"],
        ),
        row=1,
        col=1,
    )
    fig.append_trace(
        go.Scatter(
            x=cameras["timestamp"],
            y=cameras["roll"],
        ),
        row=2,
        col=1,
    )
    fig.append_trace(
        go.Scatter(
            x=cameras["timestamp"],
            y=cameras["heading"],
        ),
        row=3,
        col=1,
    )

    fig.update_layout(height=600, width=1000, title_text="Stacked Subplots")
    if show:
        fig.show()

    return fig


def plot_vertical(cameras: pl.DataFrame, show: bool = True) -> None:
    """TODO"""
    fig = make_subplots(rows=3, cols=1, subplot_titles=("Depth", "Height", "Altitude"))

    fig.append_trace(
        go.Scatter(
            x=cameras["timestamp"],
            y=cameras["depth"],
        ),
        row=1,
        col=1,
    )
    fig.append_trace(
        go.Scatter(
            x=cameras["timestamp"],
            y=cameras["height"],
        ),
        row=2,
        col=1,
    )
    fig.append_trace(
        go.Scatter(
            x=cameras["timestamp"],
            y=cameras["altitude"],
        ),
        row=3,
        col=1,
    )

    fig.update_layout(height=600, width=1000, title_text="Stacked Subplots")
    if show:
        fig.show()

    return fig


def plot_geolocations(cameras: pl.DataFrame, show: bool = True) -> None:
    """TODO"""
    fig = px.scatter_mapbox(
        cameras,
        lat="latitude",
        lon="longitude",
        color="timestamp",
        size_max=15,
        zoom=17,
        height=600,
        hover_data={"heading", "stereo_left_label", "depth"},
    )

    fig.update_layout(height=500, width=1000, mapbox_style="open-street-map")
    fig.show()
    return fig

### Plot attitude and vertical timeseries, and geolocations

In [ ]:
figs["renav_attitudes"] = plot_attitudes(cameras["updated"])
figs["vertical"] = plot_vertical(cameras["updated"])
figs["updated"] = plot_geolocations(cameras["updated"])

### Write updated cameras to file

In [ ]:
do_export: bool = True
if do_export:
    error: Optional[str] = cameras["updated"].write_csv(output_path)
    if error:
        logger.error(error)
    else:
        logger.info(f"saved file to: {output_path}")